In [41]:
import glob
import geopandas as gpd
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from libpysal.graph import read_parquet
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler
import shapely
from numba import njit
from core.cluster_validation import generate_neigbhourhood_groups, colored_crosstab
from core.utils import used_keys

from palettable.colorbrewer.qualitative import Set3_12
from sklearn.metrics import davies_bouldin_score

In [3]:
region_id = 69300

tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
chars_dir = "/data/uscuni-ulce/processed_data/chars/"
graph_dir = "/data/uscuni-ulce/processed_data/neigh_graphs/"

# region_id = 'freiburg'
# buildings_dir = streets_dir = enclosures_dir = tessellations_dir = graph_dir = '../data/freiburg/'
# chars_dir = '../data/freiburg/chars/'
# cluster_dir = '/data/uscuni-ulce/processed_data/clusters/'

In [5]:
data = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')
graph1 = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")
tessellation = gpd.read_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
)

In [ ]:
k=3
n_splits=10
df = data.copy()
graph = graph1.copy()
centroids = shapely.get_coordinates(tessellation.representative_point())
kernel='gaussian'

In [7]:
@njit
def _interpolate(weights, group):
    q = (50,)
    nan_tracker = np.isnan(group)
    if nan_tracker.all():
        return np.array([float(np.nan) for _ in q])
    group = group[~nan_tracker]
    sorter = np.argsort(group)
    group = group[sorter]
    weights = weights[~nan_tracker][sorter]

    xs = np.cumsum(weights) - 0.5 * weights
    xs = xs / weights.sum()
    ys = group
    interpolate = np.interp(
        [x / 100 for x in q],
        xs,
        ys,
    )
    return interpolate

In [8]:
@numba.njit(parallel=True)
def partial_weighted_percentile(rows, cols, partial_vals, centroids, kernel):
    """rows are the re-mapped focals, cols are re-mapped neighbours"""
    output_vals = 1
    ngroups = len(np.unique(rows))
    nrows = rows.shape[0]
    result = np.empty((ngroups, partial_vals.shape[1] * output_vals))

    istart = 0
    for g in range(ngroups):
        # # find focal start
        # istart = 0
        # while istart < nrows and rows[istart] != g:
        #     istart += 1

        # find neighbors
        iend = istart + 1
        while iend < nrows and rows[iend - 1] == rows[iend]:
            iend += 1

        neighbours = centroids[cols[istart:iend], :]
        focals = centroids[rows[istart:iend], :]
        weights = np.sqrt(((neighbours - focals)**2).sum(axis=1))
        
        not_zero = weights != 0

        if kernel == 'gausian':
            u = weights / np.max(weights)
            weights = np.exp(-((u / 2) ** 2)) / (np.sqrt(2) * np.pi)
        elif kernel == 'inverse':
            weights = 1 / weights
        else:
            # default - reverse weights
            weights = 0 - weights
        
        
        ## for every column apply iqr and percentiles
        for c in numba.prange(partial_vals.shape[1]):
            
            col_vals = partial_vals[cols[istart:iend], c]
            res_index = output_vals * c

            if np.isnan(col_vals).all():
                result[g, res_index] = np.nan
                continue

            else:
                res = _interpolate(weights[not_zero], col_vals[not_zero])
                result[g, res_index] = res[0]

        # # go to next group
        istart = iend
    return result




In [21]:
%%time

def spatially_weighted_partial_lag(df, graph, centroids, kernel, k, n_splits):

    A = graph.transform("B").sparse
    ids = graph.unique_ids.values
    rows = np.arange(A.shape[0])
    values = df.values
    
    final_result = pd.DataFrame(
        np.empty((values.shape[0], values.shape[1])), index=ids
    )
    
    for source in np.array_split(rows, n_splits):
        Q = A[source, :].copy()
        for _ in range(1, k):
            next_step = Q @ A
            Q += next_step
    
        sparray = Q.tocoo(copy=False)
    
        unique_tail = np.unique(sparray.col)
        cols_dict = pd.Series(np.arange(len(unique_tail)), index=unique_tail)
        columns_to_pass = cols_dict.loc[sparray.col].values
        rows_to_pass = cols_dict.loc[source[sparray.row]].values
    
        partial_vals = values[unique_tail, :]
        partial_centroids = centroids[unique_tail, :]
    
        partial_res = partial_weighted_percentile(
                rows_to_pass, columns_to_pass, partial_vals, partial_centroids, kernel
            )
    
        final_result.iloc[source, :] = partial_res

    final_result.columns = [c + "_median" for c in df.columns]
    return final_result

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.01 μs


In [22]:
final_result = spatially_weighted_partial_lag(df, graph, centroids, kernel, k, n_splits)

In [23]:
final_result

,sdbAre_median,sdbPer_median,sdbCoA_median,ssbCCo_median,ssbCor_median,ssbSqu_median,ssbERI_median,ssbElo_median,ssbCCM_median,ssbCCD_median,...,ldkAre_median,ldkPer_median,lskCCo_median,lskERI_median,lskCWA_median,ltkOri_median,ltkWNB_median,likWBB_median,sdsAre_median,likWCe_median
-1933,221.860722,61.556303,0.0,0.558163,6.0,2.838692,0.986193,0.766325,9.700094,1.059572,...,11796.568426,478.821626,0.377123,0.896509,98.392520,5.557114,0.016356,0.238336,9508.422689,0.001048
-1932,222.701313,61.542418,0.0,0.551600,6.0,2.131134,0.986215,0.728880,9.777312,1.239782,...,11796.568426,463.762380,0.377123,0.900994,70.369453,5.434029,0.017024,0.303245,9508.422689,0.001246
-1931,225.847161,62.232339,0.0,0.558174,6.0,3.392867,0.984239,0.766803,9.742882,1.256452,...,11796.568426,463.762380,0.377599,0.896509,70.369453,5.557114,0.016745,0.299849,9339.660781,0.001048
-1930,222.767739,61.567831,0.0,0.545340,6.0,4.604025,0.986102,0.741135,9.894116,1.130469,...,13856.458866,591.026872,0.363101,0.886402,172.129599,5.283687,0.014067,0.110245,3115.301516,0.001048
-1929,4122.980240,272.281446,0.0,0.512760,4.0,0.180687,0.998587,0.535814,45.014492,1.484694,...,186488.021458,3532.199541,0.143255,0.520180,1331.316861,40.735333,0.002831,0.131894,265625.489921,0.000086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299059,77.926151,38.950199,0.0,0.477424,4.0,0.754631,0.999725,0.460186,7.443199,0.098218,...,28064.447793,680.579552,0.493020,1.042630,81.789059,29.828648,0.002939,0.045659,108672.023661,0.000392
299060,75.232536,37.453995,0.0,0.501608,4.0,1.596611,1.000098,0.529393,6.756091,0.160605,...,28064.447793,680.579552,0.493020,1.042630,81.789059,29.828648,0.002939,0.045659,108672.023661,0.000392
299061,75.235089,37.817818,0.0,0.493782,4.0,0.876952,1.000098,0.480733,7.047278,0.117632,...,28064.447793,680.579552,0.493020,1.042630,81.789059,29.828648,0.002939,0.045659,108672.023661,0.000392
299062,93.110513,42.595089,0.0,0.489356,4.0,0.659523,0.998560,0.506422,7.415875,0.040665,...,28064.447793,680.579552,0.493020,1.042630,81.789059,29.828648,0.002939,0.045659,108672.023661,0.000392


In [24]:
data

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,ssbCCM,ssbCCD,...,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB,sdsAre,likWCe
-1933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12331.629405,517.944880,0.323345,0.955507,118.499644,9.199255,0.015446,NaN,55962.996535,0.000081
-1932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,435.260502,274.068246,0.030510,0.730177,159.106057,9.709908,0.021892,NaN,55962.996535,0.002297
-1931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1084.359120,212.010299,0.162748,0.738946,81.720059,4.396669,0.037734,NaN,2422.191399,0.000922
-1930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,138.230998,50.222715,0.475208,0.958111,7.628495,12.022472,0.079645,NaN,1547.125907,0.007234
-1929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7659.205935,457.005065,0.233216,0.919543,140.376436,41.802499,0.010941,NaN,23915.039836,0.000131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299059,75.225865,36.503153,0.0,0.523964,4.0,0.308174,0.998871,0.529423,6.757689,0.012755,...,28064.447793,680.579552,0.493020,1.042630,81.789059,29.828648,0.002939,0.045659,108672.023661,0.000392
299060,99.143049,44.524838,0.0,0.424405,4.0,0.197566,0.998745,0.384297,8.621600,0.013901,...,28064.447793,680.579552,0.493020,1.042630,81.789059,29.828648,0.002939,0.045659,108672.023661,0.000392
299061,208.004116,66.440793,0.0,0.476146,6.0,9.118904,0.885149,0.674641,9.596795,2.791236,...,28064.447793,680.579552,0.493020,1.042630,81.789059,29.828648,0.002939,0.045659,108672.023661,0.000392
299062,75.241771,38.770296,0.0,0.404768,4.0,4.757553,1.003082,0.377719,7.484418,0.274223,...,28064.447793,680.579552,0.493020,1.042630,81.789059,29.828648,0.002939,0.045659,108672.023661,0.000392


In [14]:
final_result.to_parquet(f'/data/uscuni-ulce/processed_data/context_data/unprocessed_context_chars_{region_id}_lag_{k}_sw.parquet')

In [25]:
final_result = spatially_weighted_partial_lag(df, graph, centroids, 'inverse', k, n_splits)

In [26]:
%%time
higher = graph1.higher_order(k=3, lower_order=True)
# transform
# percentile

CPU times: user 18.4 s, sys: 1.85 s, total: 20.2 s
Wall time: 20.2 s


In [27]:
%%time

from shapely import distance
centroids = tessellation.representative_point()

def _distance_decay_weights(group):
    focal = group.index[0][0]
    neighbours = group.index.get_level_values(1)
    distances = distance(centroids.loc[focal], centroids.loc[neighbours])
    distance_decay = 1 / distances
    return distance_decay.values

decay_graph = higher.transform(_distance_decay_weights)

CPU times: user 2min 40s, sys: 46.9 ms, total: 2min 40s
Wall time: 2min 40s


In [28]:
import momepy as mm

In [30]:
res = mm.percentile(data['sdbAre'], decay_graph, q=[50])

In [32]:
data['sdbAre']

-1933             NaN
-1932             NaN
-1931             NaN
-1930             NaN
-1929             NaN
              ...    
 299059     75.225865
 299060     99.143049
 299061    208.004116
 299062     75.241771
 299063    116.559504
Name: sdbAre, Length: 300997, dtype: float64

In [33]:
res

,50
focal,
-1933,210.208007
-1932,163.100607
-1931,215.135215
-1930,179.147311
-1929,2261.766174
...,...
299059,121.391606
299060,104.184452
299061,90.153414


In [34]:
final_result.iloc[:, 0]

-1933       210.208007
-1932       163.100607
-1931       215.135215
-1930       179.147311
-1929      2261.766174
              ...     
 299059     121.391606
 299060     104.184452
 299061      90.153414
 299062      89.468522
 299063      85.371591
Name: sdbAre_median, Length: 300997, dtype: float64

In [36]:
higher.describe(data['sdbAre'], statistics=['mean'])['mean']

focal
-1933       278.624281
-1932       291.339485
-1931       306.293042
-1930       258.910183
-1929      3957.084643
              ...     
 299059     120.616620
 299060     118.224902
 299061     107.338795
 299062     120.615029
 299063     116.483256
Name: mean, Length: 300997, dtype: float64

In [37]:
from pandas.testing import assert_series_equal

In [39]:
isolates = graph1.assign_self_weight(0).isolates

In [40]:
assert_series_equal(res.drop(isolates)[50], final_result.drop(isolates).iloc[:, 0], check_names=False)

In [281]:
not_equal = res[50] != final_result.iloc[:, 0]

In [282]:
not_equal = not_equal[not_equal].index.values

In [289]:
graph1.cardinalities[not_equal]

focal
-643       1
 554       2
 785       1
 847       2
 1097      2
          ..
 294534    1
 298001    2
 298036    2
 298531    1
 299037    1
Name: cardinalities, Length: 241, dtype: int64

In [303]:
final_result.loc[554, 'sdbAre_median'], res.loc[554, 50],

(nan, nan)

In [304]:
graph1[554]

neighbor
-1039    1
 554     1
Name: weight, dtype: int64

In [298]:
np.isin(isolates, not_equal).all()

True

In [300]:
final_result.loc[isolates, 'sdbAre_median']

focal
-643      NaN
 785      NaN
 1144     NaN
 1276     NaN
 1542     NaN
           ..
 220333   NaN
 293066   NaN
 294534   NaN
 298531   NaN
 299037   NaN
Name: sdbAre_median, Length: 163, dtype: float64

In [302]:
not_equal[~np.isin(not_equal, isolates)]

array([   554,    847,   1097,   1619,   2218,   3050,   3145,   3177,
         3810,   4082,   4172,   5907,   7987,   8479,   9928,  15250,
        17362,  17513,  17596,  17719,  18349,  18372,  18424,  18510,
        23828,  25453,  25456,  26876,  27376,  27760,  29130,  29421,
        29666,  29936,  30296,  33783,  33803,  39845,  40536,  41345,
        41626,  42220,  43636,  54363,  54642,  55009,  56605,  61686,
        61689,  66609,  67692,  67741,  72972,  75580,  76469,  79025,
        79026,  80612,  80824,  84456,  96796,  96841,  96934,  97452,
       104147, 104238, 104392, 105174, 115558, 126136, 138833, 138836,
       143380, 152700, 221878, 225867, 298001, 298036])